<a href="https://colab.research.google.com/github/dkyos/data-analysis/blob/master/Train_Your_Own_Q%26A_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![DoctorBert](https://snag.gy/oF84Gk.jpg)


![DoctorBert](https://i.ytimg.com/vi/nPemP-Q0Xn8/hqdefault.jpg)

This is a Colab Demo of our DocProduct Tensorflow 2.0 Hackathon Project

Project details can be seen on our Github repo
https://github.com/Santosh-Gupta/DocProduct

and our Devpost page
https://devpost.com/software/nlp-doctor

This notebook shows how you can use our code to train your own Q&A model but inserting your own data. 

The model architecture that is being trained is shown below

![DoctorBert](https://i.imgur.com/IRCyKIL.jpg?1)


In [0]:
#@title Install Faiss, TF 2.0, and our Github. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import tensorflow.keras.backend as K
!pip install https://github.com/Santosh-Gupta/DocProduct/archive/master.zip
    
import argparse
import os
import requests



--2019-05-10 03:36:55--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20190510T033656Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=AgoJb3JpZ2luX2VjEHIaCXVzLWVhc3QtMSJGMEQCIECtnkIy%2FeXVbJUBVmYMIPLpNNxgA5hw0dsf%2BGlRs4%2B8AiAtXT3G5jBUofVTxShzKuwumeyaWWkHKlfHiK7MhQ2BJSraAwhrEAAaDDQ1NTg2NDA5ODM3OCIMLJKTDy8nMta%2F%2F1ozKrcDBedHK

In [0]:
# Download pre-trained Bert. This version downloads the bioBert pretrained model but you can set it to download any Bert pretrained model. 

import urllib.request

urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_pubmed_pmc.tar.gz', 'BioBert.tar.gz') #Link to pre-trained BERT

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

os.listdir('BioBertFolder/pubmed_pmc_470k')

In [0]:
#@title Download our fine-tuned model (use is optional). . Double Click to see code

import urllib.request

urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

os.listdir('BioBertFolder/pubmed_pmc_470k')

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if not os.path.exists('models'):
    os.mkdir('models')
    
if not os.path.exists('models/bertffn_crossentropy'):
    os.mkdir('models/bertffn_crossentropy')
    
if not os.path.exists('models/bertffn_crossentropy/bertffn'):
    os.mkdir('models/bertffn_crossentropy/bertffn')
    
import requests

if not os.path.exists('qa_embeddings'):
    os.mkdir('qa_embeddings')
    
if not os.path.exists('chkpoint'):
    os.mkdir('chkpoint')
    
file_id = '1thX75_cMkly5btgxTydgV6YDKnrUczZs'

download_file_from_google_drive(file_id, 'models/bertffn_crossentropy/bertffn/bertffn.data-00000-of-00001')

file_id = '11q29T38EysVPueD1PMQzZoKs5QFr3jRT'

download_file_from_google_drive(file_id, 'models/bertffn_crossentropy/bertffn/bertffn.index')

file_id = '1iI5Aow_7pQSmvpxGnMVKw9OJV-nckkxz'

download_file_from_google_drive(file_id, 'models/bertffn_crossentropy/bertffn/bertffn.checkpoint')

os.listdir('models/bertffn_crossentropy/bertffn')

if not os.path.exists('FFNNFolder'):
    os.makedirs('FFNNFolder')
    
file_id = '1-258Wlpp5UwwqCPezy6DRYBozO4wBUBw'

download_file_from_google_drive(file_id, 'FFNNFolder/data-00000-of-00001')

file_id = '1-9JmtJJ_XGV0wClZxs2Px4hDL-Pi9YA_'

download_file_from_google_drive(file_id, 'FFNNFolder/index')

file_id = '1-5hczUjQfCTpBg1HhpLrgXFTkve1xuRM'

download_file_from_google_drive(file_id, 'FFNNFolder/checkpoint')

#download data

# file_id = '1blkZdV1BNWesD0mJ6Pm1H1IhCMYThkWJ'

# download_file_from_google_drive(file_id, 'Float16Embeddings.pkl')

if not os.path.exists('data'):
    os.mkdir('data')
    
file_id = '1G-NhMv_AQli-o1ES3lJS4LlOhRP59NT8'  #Sample data

download_file_from_google_drive(file_id, 'data/sampleData.csv')

if not os.path.exists('newModels'):
    os.mkdir('newModels')

In [0]:
#Download sample data from google drive. 

if not os.path.exists('data'):
    os.mkdir('data')

#Google drive links looks like https://drive.google.com/open?id=1G-NhMv_AQli-o1ES3lJS4LlOhRP59NT8

file_id = '1vZR7C45vQG4_SFHMJfmq9d2QK8Zgi8Dq'  #only put file ID from Gdrive link 

download_file_from_google_drive(file_id, 'data/sampleData.csv')

#look at data in pandas

import pandas as pd

dataPandas = pd.read_csv('data/sampleData.csv')

#This is how the data should be formatted, including all lower question column names. 
dataPandas.head()

,question,answer
0,Trying to get y friend to see a doctor but she...,This is most likely a Bartholin's Gland Cyst. ...
1,Ease my worries pleaseMy husband is about to s...,Would be helpful with the exact values of the ...
2,"Severe shoulder, neck and back pain after teta...",I seriously don't think your pain derives from...
3,Knee problemHere's the run down. This past we...,It's possible that you caused some damage whil...
4,Had a strange experience last night.Woke up in...,Sounds like you had an episode of positional h...


In [0]:
#Import relevant functions and objects

from Scripts.dataset import create_dataset_for_bert
from Scripts.models import MedicalQAModelwithBert
from Scripts.loss import qa_pair_loss, qa_pair_cross_entropy_loss
from Scripts.tokenization import FullTokenizer
from Scripts.metrics import qa_pair_batch_accuracy

#Set parameters and paths

model_path='newModels' #to train brand new model, or use our pretrained model by uncommenting the line below
# model_path='models/bertffn_crossentropy/bertffn/bertffn'

data_path='data/' #This is where you put your own data!

num_epochs=1
num_gpu=1
batch_size=64
learning_rate=2e-5
validation_split=0.2
loss='categorical_crossentropy',
pretrained_path='BioBertFolder/pubmed_pmc_470k/'
max_seq_len=256  # Max is 512

if loss == 'categorical_crossentropy':
    loss_fn = qa_pair_cross_entropy_loss
else:
    loss_fn = qa_pair_loss
    
K.set_floatx('float32')

tokenizer = FullTokenizer(os.path.join(pretrained_path, 'vocab.txt'))

d = create_dataset_for_bert(
    data_path, tokenizer=tokenizer, batch_size=batch_size,
    shuffle_buffer=500000, dynamic_padding=True, max_seq_length=max_seq_len)

eval_d = create_dataset_for_bert(
    data_path, tokenizer=tokenizer, batch_size=batch_size,
    mode='eval', dynamic_padding=True, max_seq_length=max_seq_len,
    bucket_batch_sizes=[64, 64, 64])

medical_qa_model = MedicalQAModelwithBert(
    config_file=os.path.join(
        pretrained_path, 'bert_config.json'),
    checkpoint_file=os.path.join(pretrained_path, 'biobert_model.ckpt'))

optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

medical_qa_model.compile(
    optimizer=optimizer, loss=loss_fn, metrics=[qa_pair_batch_accuracy])

epochs = num_epochs

loss_metric = tf.keras.metrics.Mean()

medical_qa_model.fit(d, epochs=epochs)
medical_qa_model.summary()
medical_qa_model.save_weights(model_path)
medical_qa_model.evaluate(eval_d)

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


TF Record not found
Converting file data/sampleData.csv to TF Record
   3329/Unknown - 3156s 948ms/step - loss: 1.8336 - qa_pair_batch_accuracy: 0.4226